### Questions
- Length of full vectors assumed to be 5000?

In [3]:
# coding: utf-8

import gensim
import math
from copy import copy

import sys
from collections import OrderedDict
import time
import pickle
import re

In [4]:
# Fix paths
import os
print os.getcwd()  # '/Users/sipola/Google Drive/education/coursework/graduate/edinburgh/nlu/nlu-assignment1'
os.chdir('/Users/sipola/Google Drive/education/coursework/graduate/edinburgh/nlu/nlu-assignment1')
print os.getcwd()

/Users/sipola/Google Drive/education/coursework/graduate/edinburgh/nlu/nlu-assignment1
/Users/sipola/Google Drive/education/coursework/graduate/edinburgh/nlu/nlu-assignment1


In [5]:
try:
    os.makedirs('run')
except OSError:
    pass

In [6]:
arg1 = 'a'
arg2 = 'data/vocabulary.txt'
arg3 = 'data/word_contexts.txt'

In [7]:
# these are indices for house, home and time in the data. Don't change.
house_noun = 80
home_noun = 143
time_noun = 12

In [8]:
'''
(f) helper class, do not modify.
provides an iterator over sentences in the provided BNC corpus
input: corpus path to the BNC corpus
input: n, number of sentences to retrieve (optional, standard -1: all)
'''
class BncSentences:
    def __init__(self, corpus, n=-1):
        self.corpus = corpus
        self.n = n

    def __iter__(self):
        n = self.n
        ret = []
        for line in open(self.corpus):
            line = line.strip().lower()
            if line.startswith("<s "):
                ret = []
            elif line.strip() == "</s>":
                if n > 0:
                    n -= 1
                if n == 0:
                    break
                yield copy(ret)
            else:
                parts = line.split("\t")
                if len(parts) == 3:
                    word = parts[-1]
                    idx = word.rfind("-")
                    word, pos = word[:idx], word[idx+1:]
                    if word in ['thus', 'late', 'often', 'only', 'usually', 'however', 'lately', 'absolutely', 'hardly', 'fairly', 'near', 'similarly', 'sooner', 'there', 'seriously', 'consequently', 'recently', 'across', 'softly', 'together', 'obviously', 'slightly', 'instantly', 'well', 'therefore', 'solely', 'intimately', 'correctly', 'roughly', 'truly', 'briefly', 'clearly', 'effectively', 'sometimes', 'everywhere', 'somewhat', 'behind', 'heavily', 'indeed', 'sufficiently', 'abruptly', 'narrowly', 'frequently', 'lightly', 'likewise', 'utterly', 'now', 'previously', 'barely', 'seemingly', 'along', 'equally', 'so', 'below', 'apart', 'rather', 'already', 'underneath', 'currently', 'here', 'quite', 'regularly', 'elsewhere', 'today', 'still', 'continuously', 'yet', 'virtually', 'of', 'exclusively', 'right', 'forward', 'properly', 'instead', 'this', 'immediately', 'nowadays', 'around', 'perfectly', 'reasonably', 'much', 'nevertheless', 'intently', 'forth', 'significantly', 'merely', 'repeatedly', 'soon', 'closely', 'shortly', 'accordingly', 'badly', 'formerly', 'alternatively', 'hard', 'hence', 'nearly', 'honestly', 'wholly', 'commonly', 'completely', 'perhaps', 'carefully', 'possibly', 'quietly', 'out', 'really', 'close', 'strongly', 'fiercely', 'strictly', 'jointly', 'earlier', 'round', 'as', 'definitely', 'purely', 'little', 'initially', 'ahead', 'occasionally', 'totally', 'severely', 'maybe', 'evidently', 'before', 'later', 'apparently', 'actually', 'onwards', 'almost', 'tightly', 'practically', 'extremely', 'just', 'accurately', 'entirely', 'faintly', 'away', 'since', 'genuinely', 'neatly', 'directly', 'potentially', 'presently', 'approximately', 'very', 'forwards', 'aside', 'that', 'hitherto', 'beforehand', 'fully', 'firmly', 'generally', 'altogether', 'gently', 'about', 'exceptionally', 'exactly', 'straight', 'on', 'off', 'ever', 'also', 'sharply', 'violently', 'undoubtedly', 'more', 'over', 'quickly', 'plainly', 'necessarily']:
                        pos = "r"
                    if pos == "j":
                        pos = "a"
                    ret.append(gensim.utils.any2unicode(word + "." + pos))

In [9]:
sparse1 = [(0,1), (2,1), (4,2)]
sparse2 = [(0,1), (1,2), (4,1)]
sparse3 = []
full1 = [1, 0, 1, 0, 2]
full2 = [1, 2, 0, 0, 1]
full3 = [0, 0, 0, 0, 0]

def is_sparse(vector):
    return not vector or isinstance(vector[0], tuple)

def sparse2full(sparse, length=5000):
    sparse = list(sparse)
    if not is_sparse(sparse):
        return sparse
    if length is None:
        length = max([tup[0] for tup in sparse]) + 1
    full = [0] * length
    for tup in sparse:
        full[tup[0]] = tup[1]
    return full

def full2sparse(full):
    if is_sparse(full):
        return full
    sparse = [(c, n) for c, n in enumerate(full) if n > 0]
    return sparse

print is_sparse(sparse1)
print is_sparse(sparse3)
print is_sparse(full1)

print sparse2full(sparse1, 5)
print sparse2full(full1, 5)
print sparse2full(sparse2, 5)
print sparse2full(sparse3, 5)

print full2sparse(full1)
print full2sparse(sparse1)
print full2sparse(full2)
print full2sparse(full3)

True
True
False
[1, 0, 1, 0, 2]
[1, 0, 1, 0, 2]
[1, 2, 0, 0, 1]
[0, 0, 0, 0, 0]
[(0, 1), (2, 1), (4, 2)]
[(0, 1), (2, 1), (4, 2)]
[(0, 1), (1, 2), (4, 1)]
[]


In [27]:
'''
(a) function load_corpus to read a corpus from disk
input: vocabFile containing vocabulary
input: contextFile containing word contexts
output: id2word mapping word IDs to words
output: word2id mapping words to word IDs
output: vectors for the corpus, as a list of sparse vectors
'''
def load_corpus(vocabFile, contextFile):
    id2word = {}
    word2id = {}
    vectors = []

    with open(vocabFile, 'r') as f:
        vocab = f.read().splitlines()
    
    with open(contextFile, 'r') as f:
        context = f.read().splitlines()
    
    for big_string in context:
        tups = []
        if big_string[0] != '0':  # words without context words have a string that starts with '0'
            cns = big_string.split(' ')[1:]
            for cn in cns:
                c, n = cn.split(':')
                tups.append((int(c), int(n)))
        vectors.append(tups)
    
    for ID, word in enumerate(vocab):
        id2word[ID] = word
        word2id[word] = ID

    return id2word, word2id, vectors

In [28]:
# this can give you an indication whether part a (loading a corpus) works.
# not guaranteed that everything works.
print("(a): load corpus")
try:
    id2word, word2id, vectors = load_corpus(arg2, arg3)
    if not id2word:
        print("\tError: id2word is None or empty")
        exit()
    if not word2id:
        print("\tError: id2word is None or empty")
        exit()
    if not vectors:
        print("\tError: id2word is None or empty")
        exit()
    print("\tPass: load corpus from file")
except Exception as e:
    print("\tError: could not load corpus from disk")
    print(e)

try:
    if not id2word[house_noun] == "house.n" or not id2word[home_noun] == "home.n" or not id2word[time_noun] == "time.n":
        print("\tError: id2word fails to retrive correct words for ids")
    else:
        print("\tPass: id2word")
except Exception:
    print("\tError: Exception in id2word")
    print(e)

try:
    if not word2id["house.n"] == house_noun or not word2id["home.n"] == home_noun or not word2id["time.n"] == time_noun:
        print("\tError: word2id fails to retrive correct ids for words")
    else:
        print("\tPass: word2id")
except Exception:
    print("\tError: Exception in word2id")
    print(e)

(a): load corpus
	Pass: load corpus from file
	Pass: id2word
	Pass: word2id


In [32]:
id2word, word2id, vectors = load_corpus('data/vocabulary.txt', 'data/word_contexts.txt')

In [13]:
'''
(b) function cosine_similarity to calculate similarity between 2 vectors
input: vector1
input: vector2
output: cosine similarity between vector1 and vector2 as a real number
'''
def cosine_similarity(vector1, vector2):
    
    # Make sure inputs are full.
    vector1 = sparse2full(vector1)
    vector2 = sparse2full(vector2)
    
    def dot_prod(v1, v2):
        return sum([x * y for x, y in zip(v1, v2)])

    num = dot_prod(vector1, vector2)
    denom = dot_prod(vector1, vector1)**0.5 * dot_prod(vector2, vector2)**0.5
    
    return num / float(denom)

In [14]:
cosine_similarity([1, 0], [1, 1])

0.7071067811865475

In [15]:
# this can give you an indication whether part b (cosine similarity) works.
# these are very simple dummy vectors, no guarantee it works for our actual vectors.
import numpy
print("(b): cosine similarity")
try:
    cos = cosine_similarity([(0,1), (2,1), (4,2)], [(0,1), (1,2), (4,1)])
    if not numpy.isclose(0.5, cos):
        print("\tError: sparse expected similarity is 0.5, was {0}".format(cos))
    else:
        print("\tPass: sparse vector similarity")
except Exception:
    print("\tError: failed for sparse vector")
try:
    cos = cosine_similarity([1, 0, 1, 0, 2], [1, 2, 0, 0, 1])
    if not numpy.isclose(0.5, cos):
        print("\tError: full expected similarity is 0.5, was {0}".format(cos))
    else:
        print("\tPass: full vector similarity")
except Exception:
    print("\tError: failed for full vector")

(b): cosine similarity
	Pass: sparse vector similarity
	Pass: full vector similarity


In [41]:
def make_test_word_list():
    w1 = (80, 'house.n')
    w2 = (143, 'home.n')
    w3 = (12, 'time.n')
    return [w1, w2, w3]

In [44]:
def test_words(vectors, words_to_test = make_test_word_list()):
    for i, tup1 in enumerate(words_to_test):
        for j, tup2 in enumerate(words_to_test):
            if j > i:
                id1, str1 = tup1
                id2, str2 = tup2
                sim = cosine_similarity(vectors[id1], vectors[id2])
                print 'similarity between {} and {}: {}'.format(str1, str2, sim)
    return None

In [45]:
test_words(vectors)

similarity between house.n and home.n: 0.812743856464
similarity between house.n and time.n: 0.82359219053
similarity between home.n and time.n: 0.818144793373


In [18]:
'''
(d) function tf_idf to turn existing frequency-based vector model into tf-idf-based vector model
input: freqVectors, a list of frequency-based vectors
output: tfIdfVectors, a list of tf-idf-based vectors
'''
def tf_idf(freqVectors):
    
    tfIdfVectors = []
    N = len(freqVectors)
    
    # Convert to sparse vectors.
    freqVectors = [full2sparse(fv) for fv in freqVectors]
    
    # Define function that calculates TF-IDF for a single term.
    def tf_idf_word(tf, N, df):
        return (1 + math.log(tf, 2)) * (1 + math.log(N / float(df), 2))

    # Generate dictionary df (document frequency of term). Do this by iterating over documents.
    df_dict = {}
    for fv in freqVectors:
        for term, _ in fv:  # don't need frequency
            df_dict[term] = df_dict.get(term, 0) + 1
        
    for fv in freqVectors:
        tfIdfVectors.append([(term, tf_idf_word(tf, N, df_dict[term])) for term, tf in fv])

    return tfIdfVectors

In [19]:
sparse1 = [(0,1), (2,1), (4,2)]
sparse2 = [(0,1), (1,2), (4,1)]
sparse3 = [(0,1), (2,1), (4,1)]
full1 = [1, 0, 1, 0, 2]
full2 = [1, 2, 0, 0, 1]

vectors_test = [full1, sparse2, sparse3]

# for i, fv in enumerate(vectors_test):
#     if not is_sparse(fv):
#         vectors_test[i] = full2sparse(fv)

print vectors_test

tf_idf(vectors_test)

[[1, 0, 1, 0, 2], [(0, 1), (1, 2), (4, 1)], [(0, 1), (2, 1), (4, 1)]]


[[(0, 1.0), (2, 1.584962500721156), (4, 2.0)],
 [(0, 1.0), (1, 5.169925001442312), (4, 1.0)],
 [(0, 1.0), (2, 1.584962500721156), (4, 1.0)]]

In [20]:
tf_idf_output = tf_idf(vectors)
print tf_idf_output[0][:10]

[(1, 46.447381754818345), (7, 60.42345848148703), (4, 48.012182361527785), (3, 47.77588941708801), (6, 54.736678364323744), (20, 55.45621663800088), (2, 43.09843681189252), (8, 47.77353357204966), (10, 53.73375562076166), (9, 53.766808961711504)]


In [21]:
test_words(tf_idf_output)

similarity between house.n and home.n: 0.644002976484
similarity between house.n and time.n: 0.57753821538
similarity between home.n and time.n: 0.54386864522


In [22]:
# this gives you an indication whether your conversion into tf-idf space works.
# this does not test for vector values in tf-idf space, hence can't tell you whether tf-idf has been implemented correctly
print("(d) converting to tf-idf space")
id2word, word2id, vectors = load_corpus(arg2, arg3)
try:
    tfIdfSpace = tf_idf(vectors)
    if not len(vectors) == len(tfIdfSpace):
        print("\tError: tf-idf space does not correspond to original vector space")
    else:
        print("\tPass: converted to tf-idf space")
except Exception as e:
    print("\tError: could not convert to tf-idf space")
    print(e)

(d) converting to tf-idf space
	Pass: converted to tf-idf space


In [23]:
'''
(f) function word2vec to build a word2vec vector model with 100 dimensions and window size 5
'''
def word2vec(corpus, learningRate, downsampleRate, negSampling):
    
    model = gensim.models.word2vec.Word2Vec(sentences=corpus,
                                            alpha=learningRate,
                                            sample=downsampleRate,
                                            negative=negSampling,
                                            size=100,
                                            window=5)
    
    return model

In [59]:
learningRate = 0.03
downsampleRate = 0.001
negSampling = 5

model = word2vec(BncSentences('/Users/sipola/Desktop/bnc.vert', 50000),
                 learningRate,
                 downsampleRate,
                 negSampling)

In [102]:
learningRate_list = [0.01, 0.03, 0.05]
downsampleRate_list = [0., 0.00001, 0.001, 0.1]
negSampling_list = [0, 5, 10]

acc_list = []

for learningRate in learningRate_list:
    for downsampleRate in downsampleRate_list:
        for negSampling in negSampling_list:
            
            t0 = time.time()
            model = word2vec(BncSentences('/Users/sipola/Desktop/bnc.vert', 50000),
                             learningRate,
                             downsampleRate,
                             negSampling)
            t1 = time.time()
            
            acc_one = model.accuracy('accuracy_test.txt')
            right = len(acc_one[-1]['correct'])
            wrong = len(acc_one[-1]['incorrect'])
            acc = right / float(right + wrong)
            
            acc_list.append(acc)

            print 'learningRate {:.2f}, downsampleRate {:.5f}, negSampling {:2}, time {:4}, acc {:.7f}'.format(learningRate, downsampleRate, negSampling, t1-t0, acc)


learningRate 0.01, downsampleRate 0.00000, negSampling  0, time                                                                                        89.3002929688, acc 0.0000000
learningRate 0.01, downsampleRate 0.00000, negSampling  5, time                                                                                        88.3039929867, acc 0.0031780
learningRate 0.01, downsampleRate 0.00000, negSampling 10, time                                                                                        88.8333699703, acc 0.0026483
learningRate 0.01, downsampleRate 0.00001, negSampling  0, time                                                                                        88.6777968407, acc 0.0000000
learningRate 0.01, downsampleRate 0.00001, negSampling  5, time                                                                                        83.3220889568, acc 0.0000000
learningRate 0.01, downsampleRate 0.00001, negSampling 10, time                                     

In [134]:
pickle.dump(acc_list, open(os.path.join("run", "word2vec_acc_list.p"), "wb"))

In [24]:
learningRate = 0.05
downsampleRate = 0.001
negSampling = 10

t0 = time.time()
model = word2vec(BncSentences('/Users/sipola/Desktop/bnc.vert'),
                 learningRate,
                 downsampleRate,
                 negSampling)
t1 = time.time()

# Run next section to save model and print stats.

In [25]:
# Save model!
model.save('run/word2vec_model')
pickle.dump(model, open(os.path.join("run", "word2vec_model_pickle.p"), "wb"))

acc_dict = model.accuracy('data/accuracy_test.txt')
right = len(acc_dict[-1]['correct'])
wrong = len(acc_dict[-1]['incorrect'])
acc = right / float(right + wrong)            

print 'learningRate {:.2f}, downsampleRate {:.5f}, negSampling {:2}, time {:100}, acc {:.7f}'.format(learningRate, downsampleRate, negSampling, t1-t0, acc)

learningRate 0.05, downsampleRate 0.00100, negSampling 10, time                                                                                         13186.149436, acc 0.4299133


In [30]:
words_to_test = ['house.n', 'home.n', 'time.n']
for i, w1 in enumerate(words_to_test):
    for j, w2 in enumerate(words_to_test):
        if j > i:
            sim = model.similarity(w1, w2)
            print 'similarity between {} and {}: {}'.format(w1, w2, sim)

similarity between house.n and home.n: 0.575759368328
similarity between house.n and time.n: 0.191773125161
similarity between home.n and time.n: 0.380011934183


In [31]:
# you may complete this part for the second part of f (training and saving the actual word2vec model)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
print("(f2) word2vec, building full model with best parameters. May take a while.")

(f2) word2vec, building full model with best parameters. May take a while.


In [178]:
'''
(h) function lda to build an LDA model with 100 topics from a frequency vector space
input: vectors
input: wordMapping mapping from word IDs to words
output: an LDA topic model with 100 topics, using the frequency vectors
'''
def lda(vectors, wordMapping):
    model = gensim.models.ldamodel.LdaModel(vectors, id2word=wordMapping, num_topics=100, passes=10, update_every=0)
    return model

In [136]:
model_lda = lda(vectors, id2word)
model_lda.save('run/lda_model')
pickle.dump(model_lda, open(os.path.join("run", "lda_model_pickle.p"), "wb"))

In [175]:
words_to_test = ['house.n', 'home.n', 'time.n']
for i, w1 in enumerate(words_to_test):
    for j, w2 in enumerate(words_to_test):
        if j > i:
            v1 = model_lda[vectors[word2id[w1]]]
            v2 = model_lda[vectors[word2id[w2]]]
            sim = cosine_similarity(v1, v2)
            print 'similarity between {} and {}: {}'.format(w1, w2, sim)

similarity between house.n and home.n: 0.428085707761
similarity between house.n and time.n: 0.0580350221231
similarity between home.n and time.n: 0.348057003822


In [138]:
'''
(j) function get_topic_words, to get words in a given LDA topic
input: ldaModel, pre-trained Gensim LDA model
input: topicID, ID of the topic for which to get topic words
input: wordMapping, mapping from words to IDs (optional)
'''
def get_topic_words(ldaModel, topicID, topn=10):
    return ldaModel.show_topic(topicID, topn)

In [200]:
# you may complete this part to get answers for part j (topic words in your LDA model)
for topic_id in range(100):
    wp = get_topic_words(model_lda, topic_id)
    word_str = ', '.join([re.sub('\..*', '', tup[0]) for tup in wp])
    print 'Topic {}: {}'.format(topic_id, word_str)

Topic 0: take, for, have, will, can, do, would, where, no, see
Topic 1: have, can, will, do, may, would, also, should, other, could
Topic 2: do, have, make, get, see, take, no, think, also, use
Topic 3: human, have, do, would, get, will, make, say, can, know
Topic 4: have, cent, do, sexual, can, will, child, no, out, would
Topic 5: more, have, very, most, so, make, become, less, will, would
Topic 6: have, president, de, attack, do, would, pool, say, go, get
Topic 7: time, dr, car, have, j, m, will, professor, nation, to
Topic 8: have, do, well, would, say, good, far, think, can, get
Topic 9: have, do, can, would, make, will, get, take, time, go
Topic 10: have, would, do, will, force, up, time, can, say, could
Topic 11: have, south, north, west, east, country, england, go, road, other
Topic 12: have, service, health, people, child, local, authority, council, member, care
Topic 13: have, will, gas, would, do, make, say, can, go, company
Topic 14: united, have, nuclear, do, new, will, blo

In [ ]:
if __name__ == '__main__':
    import sys

    part = sys.argv[1].lower()

    # these are indices for house, home and time in the data. Don't change.
    house_noun = 80
    home_noun = 143
    time_noun = 12

    # this can give you an indication whether part a (loading a corpus) works.
    # not guaranteed that everything works.
    if part == "a":
        print("(a): load corpus")
        try:
            id2word, word2id, vectors = load_corpus(sys.argv[2], sys.argv[3])
            if not id2word:
                print("\tError: id2word is None or empty")
                exit()
            if not word2id:
                print("\tError: id2word is None or empty")
                exit()
            if not vectors:
                print("\tError: id2word is None or empty")
                exit()
            print("\tPass: load corpus from file")
        except Exception as e:
            print("\tError: could not load corpus from disk")
            print(e)

        try:
            if not id2word[house_noun] == "house.n" or not id2word[home_noun] == "home.n" or not id2word[time_noun] == "time.n":
                print("\tError: id2word fails to retrive correct words for ids")
            else:
                print("\tPass: id2word")
        except Exception:
            print("\tError: Exception in id2word")
            print(e)

        try:
            if not word2id["house.n"] == house_noun or not word2id["home.n"] == home_noun or not word2id["time.n"] == time_noun:
                print("\tError: word2id fails to retrive correct ids for words")
            else:
                print("\tPass: word2id")
        except Exception:
            print("\tError: Exception in word2id")
            print(e)

    # this can give you an indication whether part b (cosine similarity) works.
    # these are very simple dummy vectors, no guarantee it works for our actual vectors.
    if part == "b":
        import numpy
        print("(b): cosine similarity")
        try:
            cos = cosine_similarity([(0,1), (2,1), (4,2)], [(0,1), (1,2), (4,1)])
            if not numpy.isclose(0.5, cos):
                print("\tError: sparse expected similarity is 0.5, was {0}".format(cos))
            else:
                print("\tPass: sparse vector similarity")
        except Exception:
            print("\tError: failed for sparse vector")
        try:
            cos = cosine_similarity([1, 0, 1, 0, 2], [1, 2, 0, 0, 1])
            if not numpy.isclose(0.5, cos):
                print("\tError: full expected similarity is 0.5, was {0}".format(cos))
            else:
                print("\tPass: full vector similarity")
        except Exception:
            print("\tError: failed for full vector")

    # you may complete this part to get answers for part c (similarity in frequency space)
    if part == "c":
        print("(c) similarity of house, home and time in frequency space")

        # your code here

    # this gives you an indication whether your conversion into tf-idf space works.
    # this does not test for vector values in tf-idf space, hence can't tell you whether tf-idf has been implemented correctly
    if part == "d":
        print("(d) converting to tf-idf space")
        id2word, word2id, vectors = load_corpus(sys.argv[2], sys.argv[3])
        try:
            tfIdfSpace = tf_idf(vectors)
            if not len(vectors) == len(tfIdfSpace):
                print("\tError: tf-idf space does not correspond to original vector space")
            else:
                print("\tPass: converted to tf-idf space")
        except Exception as e:
            print("\tError: could not convert to tf-idf space")
            print(e)

    # you may complete this part to get answers for part e (similarity in tf-idf space)
    if part == "e":
        print("(e) similarity of house, home and time in tf-idf space")

        # your code here

    # you may complete this part for the first part of f (estimating best learning rate, sample rate and negative samplings)
    if part == "f1":
        print("(f1) word2vec, estimating best learning rate, sample rate, negative sampling")

        # your code here

    # you may complete this part for the second part of f (training and saving the actual word2vec model)
    if part == "f2":
        import logging
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
        print("(f2) word2vec, building full model with best parameters. May take a while.")

        # your code here

    # you may complete this part to get answers for part g (similarity in your word2vec model)
    if part == "g":
        print("(g): word2vec based similarity")

        # your code here

    # you may complete this for part h (training and saving the LDA model)
    if part == "h":
        import logging
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
        print("(h) LDA model")

        # your code here

    # you may complete this part to get answers for part i (similarity in your LDA model)
    if part == "i":
        print("(i): lda-based similarity")

        # your code here

    # you may complete this part to get answers for part j (topic words in your LDA model)
    if part == "j":
        print("(j) get topics from LDA model")

        # your code here
